In [4]:
import hashlib
import binascii
from bip_utils import (
    Bip39SeedGenerator, Bip32Slip10Secp256k1, Bip32Slip10Ed25519,
    Bip39MnemonicGenerator, Bip39Languages, Bip39WordsNum, Bip39MnemonicValidator, Bip39Mnemonic,
    Bip44Changes, Bip84Coins, Bip84
)
from pytezos.crypto.encoding import base58_encode
from pytezos.crypto import key
from eth_account import Account
from bitcoin.wallet import CBitcoinSecret, CBech32BitcoinAddress, P2WPKHBitcoinAddress, CBitcoinAddress
from bitcoin.core.key import CECKey, CPubKey
from bitcoin.core.script import CScript,OP_0
from bitcoin.core import Hash160

# HD Wallets(public key, private key, address)

## Mnemonics

### Generate random mnemonic of twelve words

In [5]:
random_mnemonics = Bip39MnemonicGenerator(Bip39Languages.ENGLISH).FromWordsNumber(Bip39WordsNum.WORDS_NUM_12).ToStr()
random_mnemonics

'arrange film wedding ethics neck swim hover flavor host globe survey own'

### Validate Mnemonics

In [6]:
mnemonic = "invalid abandon abandon abandon abandon abandon abandon abandon abandon abandon abandon about"

# Get if a mnemonic is valid with automatic language detection, return bool
valid = Bip39MnemonicValidator(Bip39Languages.ENGLISH).IsValid(mnemonic)
# Same but specifying the language
not_valid = Bip39MnemonicValidator(Bip39Languages.ENGLISH).IsValid(random_mnemonics)

valid, not_valid

(False, True)

In [7]:
mnemonic = "imitate embody law mammal exotic transfer roof hope price swift ordinary uncle"
seed_bytes = Bip39SeedGenerator(mnemonic).Generate()

In [29]:
TEZOS_PATH = "m/44'/1729'/0'/0'"
ETHEREUM_PATH = "m/44'/60'/0'/0/0"
BITCOIN_PATH_LEGACY = "m/44'/0'/0'/0/0"
BITCOIN_PATH_SEGWIT = "m/84'/0'/0'/0/0"

### Tezos Key Store

In [9]:
bip32_ctx = Bip32Slip10Ed25519.FromSeedAndPath(seed_bytes, TEZOS_PATH)

In [10]:
private_key_hex = bip32_ctx.PrivateKey().Raw().ToHex()

In [11]:
tezos_private_key = base58_encode(bytes.fromhex(private_key_hex), prefix=b'edsk')
tezos_private_key

b'edsk4UQZnRxW6zVQdM34Mrby65gHWdqPFNisQ6wFd1gXyJ6cib6MiT'

In [12]:
account = key.Key.from_encoded_key(tezos_private_key)
tezos_address = account.public_key_hash()
tezos_address

'tz1R5X6pDpoxbMkY4AQJmUhRfEHjYN7dq66E'

### Ethereum Key Store

In [13]:
bip32_ctx = Bip32Slip10Secp256k1.FromSeedAndPath(seed_bytes, ETHEREUM_PATH)

In [14]:
private_key_hex = bip32_ctx.PrivateKey().Raw().ToHex()

In [15]:
account = Account.from_key(private_key_hex)
ethereum_private_key = account.privateKey
ethereum_private_key

HexBytes('0x396406888faded8920173dabece8976479b998429e70239b500014ed0164d6f3')

In [16]:
ethereum_address = account.address
ethereum_address

'0x5c862679fB3FF6a3a87A30a8222DF61E7eE8d841'

### Bitcoin Key Store

In [35]:
bip32_ctx = Bip32Slip10Secp256k1.FromSeedAndPath(seed_bytes, BITCOIN_PATH_SEGWIT)

In [36]:
private_key_bytes = bip32_ctx.PrivateKey().Raw().ToBytes()
private_key_bytes

b'\x92\x19\x03\xd8-.-\xf8n\x7f\x12\x18\x99I\x81:PA\xe0\x04\xbcIR\xcf#\xb2j\xaa\\\x8f\xf4\xd8'

In [37]:
account = CBitcoinSecret.from_secret_bytes(private_key_bytes)

In [38]:
script_pubkey = CScript([OP_0, Hash160(account.pub)])
bitcoin_address = CBech32BitcoinAddress.from_scriptPubKey(script_pubkey)
bitcoin_address

P2WPKHBitcoinAddress('bc1q9lcs4tv5h338dgglkm3m2clk5ca8vdvr939tk7')